Remove useless neurons

In [1]:
from settings import NET_NAME

NET_NAME

'DQN64'

In [5]:
import pandas as pd

mean_iou = {}
i = 5
res = pd.read_csv(f"../save/LunarLander-{NET_NAME}/result_{i}.csv")
res = res[res["iou"] > 0]
res = res.sort_values(by="iou", ascending=False)
mean_iou[i] = res["iou"].mean()
res.to_csv(f"../save/LunarLander-{NET_NAME}/result_{i}_parsed.csv", index=False)

find 4 outputs top connections

In [3]:
import pandas as pd

i = 5
N = 20
csv = pd.read_csv(f"../save/LunarLander-{NET_NAME}/result_{i}_parsed.csv")
output_file = f"../save/LunarLander-{NET_NAME}/output_{i}.txt"
neurons = []


def find_unique_elements(arr1, arr2, arr3, arr4):
    set1, set2, set3, set4 = set(arr1), set(arr2), set(arr3), set(arr4)
    unique1 = set1 - set2 - set3 - set4
    unique2 = set2 - set1 - set3 - set4
    unique3 = set3 - set1 - set2 - set4
    unique4 = set4 - set1 - set2 - set3

    return unique1, unique2, unique3, unique4


for label in ["w_nothing", "w_left", "w_main", "w_right"]:
    sorted_csv = csv.sort_values(by=label, ascending=False)
    top_neurons = sorted_csv.head(N)["neuron"].tolist()
    neurons.append(top_neurons)

unique_elements = find_unique_elements(neurons[0], neurons[1], neurons[2], neurons[3])
print(unique_elements, "\n")

with open(output_file, "w") as f:
    for i, label in enumerate(["w_nothing", "w_left", "w_main", "w_right"]):
        csv = csv.sort_values(by=label, ascending=False)
        top_neurons = csv.head(N)[["neuron", label]].to_records(index=False)
        print(f"\n============\n{label} top {N}:", file=f)
        for neuron, value in top_neurons:
            if neuron in unique_elements[i]:
                print(f"* {neuron} {value}", file=f)  # Special mark for unique elements
                print(f"* {neuron} {value}")
            else:
                print(f"  {neuron} {value}", file=f)
                print(f"  {neuron} {value}")

({1002, 236, 623}, {325, 555, 844, 205, 475, 606, 415}, {288, 673, 902, 652, 367, 212, 341, 316}, {609, 494, 242, 440, 24, 318}) 

  763 0.30485895
  302 0.28758878
  986 0.2645894
  643 0.25234053
  237 0.24664687
  706 0.21200533
  885 0.20753646
  279 0.20324802
  903 0.19172496
* 1002 0.16097532
  142 0.13872454
  368 0.13216262
  1007 0.104875065
  831 0.10194893
* 623 0.09671927
  488 0.09349492
* 236 0.08687392
  589 0.062169034
  468 0.061411772
  192 0.05901481
  763 0.3922268
* 555 0.3755717
  142 0.3370697
  589 0.3358096
  825 0.29140386
  706 0.22679016
  718 0.20479837
  302 0.19379257
  854 0.19233328
* 415 0.19211861
  569 0.14957115
* 205 0.14751582
  643 0.12503086
* 475 0.11712325
  167 0.10907171
* 606 0.09092694
* 325 0.081854574
  831 0.07968554
  108 0.07887353
* 844 0.067143545
* 341 0.58988327
* 212 0.55288494
  854 0.4673015
* 902 0.41630557
  885 0.30716595
  108 0.21929523
  109 0.18448631
  279 0.17271154
  368 0.14518444
* 316 0.14097175
  406 0.11672783
*

find act & inp pair

In [2]:
import numpy as np

act = np.load(f"../save/LunarLander-{NET_NAME}/actions.npy")
obs = np.load("../save/lunar.npy")

act_list = []
act_idx_list = []
max_indices = np.argmax(act, axis=1)
print(max_indices)
print(max_indices.shape)
for i in range(4):
    max_row_index = np.where(max_indices == i)[0]
    print(max_row_index.shape)
    # max_row = act[max_row_index]
    act_idx_list.append(max_row_index)
    act_list.append(obs[max_row_index])

# np.save(f'../save/LunarLander-{NET_NAME}/max_indices.npy', max_indices)

[3 3 3 ... 1 1 1]
(10000,)
(1715,)
(2127,)
(2696,)
(3462,)


## analyze specific neuron(DQN)

In [26]:
from analyze import *

ckpt_path = "../save/ckpt-mlp1024-260.pth"
data_path = "../save/lunar.npy"
print("Load model and dataset")
model, dataset = load_for_analysis(ckpt_path, data_path)
weights = model.fc3.weight.t().detach().cpu().numpy()

print("Extract features")
inputs, features, outputs = extract_feature(model, dataset)
outputs = np.array(outputs)
if not os.path.exists(settings.RESULT):
    # 如果不存在，创建路径
    os.makedirs(settings.RESULT)
np.save(os.path.join(settings.RESULT, f"actions.npy"), outputs)
print("Computing quantiles")
activations = quantile_features(features)
print("Generating atomic masks")
feat_masks = gen_feat_mask(inputs)
print(feat_masks.shape)

Load model and dataset
Extract features


100%|██████████| 157/157 [00:00<00:00, 419.39it/s]


Computing quantiles
Generating atomic masks
(10000, 16)


[x, y, Vx, Vy, angle, angular_v, l, r]

y_near_ground: <0.4

Vy_low: -0.5 ~ 0

In [39]:
def intersect(x: np.ndarray, y: np.ndarray):
    return np.array(list(set(x).intersection(set(y))))


vy_low = np.where(feat_masks[:, 7] > 0)[0]
y_near_ground = np.where(feat_masks[:, 4] > 0)[0]

vy_low_and_y_near_ground = intersect(vy_low, y_near_ground)

neuron_act = np.where(activations[:, 630] > 0)[0]
res = intersect(neuron_act, act_idx_list[0])
res = intersect(res, vy_low_and_y_near_ground)
print(res.shape)
print(act[res[10]])
print(obs[res[10]])
print(features[res[10]][630])
hook = HookLayer()
hook.hook_layer(layer=model.act2)
obs_attk = obs[res[10]]
obs_attk[2] = 1.0
print(obs_attk)
act_attk = model(torch.tensor(obs_attk, dtype=torch.float32).to(settings.DEVICE))
print(hook.features_blobs)

(1237,)
[54.300285 52.586964 50.26787  52.025227]
[ 0.30821085  0.35956365  1.         -0.44264442  0.39747119 -0.08431965
  0.          0.        ]
0.63623697
[ 0.30821085  0.35956365  1.         -0.44264442  0.39747119 -0.08431965
  0.          0.        ]
[array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)]


In [40]:
print(hook.features_blobs[0][630])
print(act_attk)

0.0
tensor([88.7608, 86.5204, 93.6968, 86.7465], device='cuda:3',
       grad_fn=<ViewBackward0>)


## analyze attack (PPO)


In [3]:
from PPO import get_PPO
import settings
from analyze import *

ckpt_path = "../save/LunarLander-PPO1024/ppo.pth"
data_path = "../save/lunar.npy"
print("Load model and dataset")
inputs, features, outputs, weight, policy = get_PPO(ckpt_path, data_path)


if not os.path.exists(settings.RESULT):
    # 如果不存在，创建路径
    os.makedirs(settings.RESULT)
np.save(os.path.join(settings.RESULT, f"actions.npy"), outputs)
print("Computing quantiles")
activations = quantile_features(features)
print("Generating atomic masks")
feat_masks = gen_feat_mask(inputs)
print(feat_masks.shape)

Load model and dataset
Computing quantiles
Generating atomic masks
(10000, 16)


In [18]:
import tianshou


def intersect(x: np.ndarray, y: np.ndarray):
    return np.array(list(set(x).intersection(set(y))))


not_l_leg = np.where(feat_masks[:, 14] == 0)[0]
r_leg = np.where(feat_masks[:, 15] > 0)[0]
x_in_centor = np.where(feat_masks[:, 0] > 0)[0]

inter = intersect(not_l_leg, r_leg)
inter = intersect(x_in_centor, inter)

neuron_act = np.where(activations[:, 212] > 0)[0]
res = intersect(neuron_act, act_idx_list[2])
res = intersect(res, inter)
print(res.shape)
print(act[res[10]])
print(obs[res[10]])
print(features[res[10]][212])
hook = HookLayer()
hook.hook_layer(policy.actor.preprocess.model.model[2])
obs_attk = obs[res[10]]
obs_attk[0] = 0.9
obs_attk[6] = 1.0
obs_attk = obs_attk.reshape(1, -1)
print(obs_attk)
print(obs_attk.shape)
act_attk = policy(tianshou.data.Batch(obs=obs_attk, info=""))
print(hook.features_blobs[0][0][212])
print(act_attk)

(14,)
[0.28012165 0.15548752 0.48146957 0.0829213 ]
[ 0.9         0.01810535  1.         -0.2070954   0.21621577 -0.67224586
  1.          1.        ]
0.37163532
[[ 0.9         0.01810535  1.         -0.2070954   0.21621577 -0.67224586
   1.          1.        ]]
(1, 8)
-1.70284
Batch(
    logits: tensor([[1.2064e-01, 8.7784e-01, 1.1213e-03, 3.9262e-04]], device='cuda:3',
                   grad_fn=<SoftmaxBackward0>),
    act: tensor([1], device='cuda:3'),
    state: None,
    dist: Categorical(probs: torch.Size([1, 4])),
)
